# Creating a voice assistant for knowledge base!

Using Whisper, OpenAI, Eleven Labs, and ActiveLoop.

## Knowledge Base

In [19]:
# Importing the necessary libraries
import os
import requests
from bs4 import BeautifulSoup
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

import re
import warnings
warnings.filterwarnings("ignore")

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ELEVEN_API_KEY = os.getenv("ELEVEN_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [5]:
pc = Pinecone(api_key=PINECONE_API_KEY)
# Initialize embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Create or get index
index_name = "voice-assistant"

# Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embeddings dimension
        metric='cosine',
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Initialize the Pinecone vector store
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
    text_key="text"
)

### 1. Web scraping content ( Python library articles ) from Hugging Face Hub

In [6]:
# Function to get the documentation URLs

def get_documentation_urls():

    return[
        	'/docs/huggingface_hub/guides/overview',
		    '/docs/huggingface_hub/guides/download',
		    '/docs/huggingface_hub/guides/upload',
		    '/docs/huggingface_hub/guides/hf_file_system',
		    '/docs/huggingface_hub/guides/repository',
		    '/docs/huggingface_hub/guides/search',
    ]

In [7]:
# Function to construct the full URL

def construct_full_url(base_url, relative_url):
    return base_url + relative_url

In [8]:
def scrape_page_content(url):
    response = requests.get(url)    #get request to the url
    soup = BeautifulSoup(response.text, 'html.parser')    #use beautiful soup to parse the html content
    text = soup.body.text.strip()     # Extract the desired content from the page (in this case, the body text)
    text = re.sub(r'[\x00-\x08\x0b-\x0c\x0e-\x1f\x7f-\xff]', '', text)
    text = re.sub(r'\s+', ' ', text)    # Remove any whitespace characters
    return text.strip()

In [9]:
# Function to scrape all content from the given URLs and save to a file

def scrape_all_content(base_url,relative_urls,filename):

    content = []
    for i in relative_urls:
        full_url = construct_full_url(base_url,i)
        scraped_content = scrape_page_content(full_url)
        content.append(scraped_content.rstrip('\n'))

    # Save the content to a file
    with open(filename, 'w', encoding='utf-8') as file:
        for item in content:
            file.write("%s\n" % item)
    return content

In [10]:
# Define a function to load documents from a file

def load_docs(root_dir,filename):
    docs = []
    try:
        loader = TextLoader(os.path.join(root_dir,filename), encoding='utf-8')
        docs.extend(loader.load_and_split())
    except Exception as e:
        pass      #if an error occurs, pass it and continue
    return docs

def split_docs(docs):
    text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap = 0)
    return text_splitter.split_documents(docs)

### 2. Embedding and storing in Pinecone


In [12]:
# define the main function

def main():
    base_url = 'https://huggingface.co'
    # Set the name of the file to which the scraped content will be saved
    filename='content.txt'
    # Set the root directory where the content file will be saved
    root_dir ='./'
    relative_urls = get_documentation_urls()

    content = scrape_all_content(base_url,relative_urls,filename)

    docs = load_docs(root_dir,filename)

    texts = split_docs(docs)

    # Extract text content and create IDs
    text_contents = [doc.page_content for doc in texts]
    doc_ids = [f"doc_{i}" for i in range(len(texts))]

    # Add documents to vector store with IDs
    vectorstore.add_texts(
        texts=text_contents,
        ids=doc_ids,
        metadatas=[doc.metadata for doc in texts]
    )

    os.remove(filename)

    print("Content scraped, embedded, and stored in Pinecone successfully!")

if __name__ == "__main__":
    main()

Content scraped, embedded, and stored in Pinecone successfully!


### 3. Voice Assistant


In [13]:
import os
import openai
import streamlit as st
from audio_recorder_streamlit import audio_recorder
from elevenlabs import generate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from streamlit_chat import message

In [14]:
# Constants
TEMP_AUDIO_PATH = "temp_audio.wav"
AUDIO_FORMAT = "audio/wav"

In [16]:
# Load environment variables from .env file and return the keys
openai.api_key = os.environ.get('OPENAI_API_KEY')
eleven_api_key = os.environ.get('ELEVEN_API_KEY')

In [17]:
# Function to load embeddings and database

def load_embeddings_and_database():
    # Initialize embeddings
    embeddings = OpenAIEmbeddings()
    
    # Initialize Pinecone
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    # Get the existing index
    db = PineconeVectorStore(
        index=pc.Index("voice-assistant"),
        embedding=embeddings,
        text_key="text"
    )
    
    return db

In [18]:
# Function that takes an audio file path and OpenAI API key as parameters

def transcribe_audio(audio_file_path,openai_key):

    # Set the OpenAI API key for authentication
    openai.api_key = openai_key
    try:
        # Open the audio file in binary read mode
        with open(audio_file_path,"rb") as audio_file:
            # Call OpenAI's Whisper API to transcribe the audio file
            response = openai.Audio.transcribe("whisper-1",audio_file)       # Uses whisper-1 model and passes the audio file
        # Return just the transcribed text from the response
        return response["text"]
    except Exception as e:
        print(f"Error calling Whisper API: {str(e)}")
        return None

The above code transcribes an audio file into text using the OpenAI Whisper API, requiring the path of the audio file and the OpenAI key as input parameters.



In [ ]:
# Record audio using audio_recorder and transcribe using transcribe_audio

def record_and_transcribe_audio():
    # Record audio from the microphone
    audio_bytes = audio_recorder()
    transcription = None
    if audio_bytes:
        st.audio(audio_bytes,format=AUDIO_FORMAT)

        with open(TEMP_AUDIO_PATH,"wb") as f:
            f.write(audio_bytes)

        if st.button("Transcribe"):
            transcription = transcribe_audio(TEMP_AUDIO_PATH,openai.api_key)
            os.remove(TEMP_AUDIO_PATH)
            display_transcription(transcription)

        return transcription
    
# Display the transcription of the audio on the app

def display_transcription(transcription):
    if transcription:
        st.write(f"Transcription: {transcription}")
        with open("audio_transcription.txt", "w+") as f:
            f.write(transcription)
    else:
        st.write("Error transcribing audio")
